In [1]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [2]:
tf.__version__

#텐서플로우 버전 2.3.0

'2.3.0'

In [3]:
device = tf.device('/gpu:0')
# 디바이스 gpu 0버전

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ln -s "/content/drive/My Drive/00.data" "/content"

ln: failed to create symbolic link '/content/00.data': File exists


In [6]:
train_path = './00.data/train/'
test_path = './00.data/test/'

#코랩기준 테스트 경로와 훈련 경로

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


x_train_datagen = ImageDataGenerator(
                rotation_range=0.2,
                rescale= 1./255.,
                zoom_range=0.2,
                horizontal_flip=True,)
y_train_datagen = ImageDataGenerator(
                rotation_range=0.2,
                rescale= 1./255.,
                shear_range=0.05,
                zoom_range=0.2,
                horizontal_flip=True,)
#
x_train_generator = x_train_datagen.flow_from_directory(
        train_path,
        classes = ['img'],
        class_mode = None,
        target_size = (128,128),
        batch_size = 32,
        seed = 1)
#
y_train_generator = y_train_datagen.flow_from_directory(
        train_path,
        classes = ['mask'],
        class_mode = None,
        target_size = (128,128),
        batch_size = 32,
        seed = 1)

trainSet_generator = zip(x_train_generator,y_train_generator)

Found 2520 images belonging to 1 classes.
Found 2520 images belonging to 1 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


In [8]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model

#first_block
def FCN32s(weight = None,input_shape = (128,128,3),initial_filter = 8):
    inputs = Input(input_shape)
    conv1 = Conv2D(initial_filter , (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1_1 = Conv2D(initial_filter , (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1_2 = Dropout(0.5)(conv1_1)
    pool1 = MaxPool2D(pool_size = (2,2))(conv1_2)
    
    conv2 = Conv2D(initial_filter * 2, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2_1 = Conv2D(initial_filter * 2, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2_2 = Dropout(0.5)(conv2_1)
    pool2 = MaxPool2D(pool_size = (2,2))(conv2_2)

    conv3 = Conv2D(initial_filter * 4, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3_1 = Conv2D(initial_filter * 4, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3_2 = Dropout(0.5)(conv3_1)
    pool3 = MaxPool2D(pool_size = (2,2))(conv3_2)

    conv4 = Conv2D(initial_filter * 8, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4_1 = Conv2D(initial_filter * 8, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4_2 = Dropout(0.5)(conv4_1)
    pool4 = MaxPool2D(pool_size = (2,2))(conv4_2)

    conv5 = Conv2D(initial_filter * 16, (3,3), activation = 'relu', padding = 'same', kernel_initializer='he_normal')(pool4)
    conv5_1 = Conv2D(initial_filter * 16, (3,3), activation = 'relu', padding = 'same', kernel_initializer='he_normal')(conv5)
    pool5 = MaxPool2D(pool_size = (2,2))(conv5_1)

    conv6 = Conv2D(initial_filter * 32, (3,3), activation = 'relu', padding = 'same', kernel_initializer='he_normal')(pool5)
    conv7 = Conv2D(initial_filter * 32, (3,3), activation = 'relu', padding = 'same', kernel_initializer='he_normal')(conv6)
    up = UpSampling2D(size=(32,32))(conv7)

    out = Conv2D(1, (1,1))(up)

    model = Model(inputs, out)
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    if weight:
        model.load_weights(weight)


    return model

In [9]:
#first_block
def FCN16s(weight = None,input_shape = (128,128,3),initial_filter = 8):
    inputs = Input(input_shape)
    conv1 = Conv2D(initial_filter , (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1_1 = Conv2D(initial_filter , (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1_2 = Dropout(0.5)(conv1_1)
    pool1 = MaxPool2D(pool_size = (2,2))(conv1_2)
    # 64 * 64

    conv2 = Conv2D(initial_filter * 2, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2_1 = Conv2D(initial_filter * 2, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2_2 = Dropout(0.5)(conv2_1)
    pool2 = MaxPool2D(pool_size = (2,2))(conv2_2)
    # 32 * 32

    conv3 = Conv2D(initial_filter * 4, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3_1 = Conv2D(initial_filter * 4, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3_2 = Dropout(0.5)(conv3_1)
    pool3 = MaxPool2D(pool_size = (2,2))(conv3_2)
    # 16 * 16
    conv4 = Conv2D(initial_filter * 8, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4_1 = Conv2D(initial_filter * 8, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4_2 = Dropout(0.5)(conv4_1)
    pool4 = MaxPool2D(pool_size = (2,2))(conv4_2)
    # 
    conv5 = Conv2D(initial_filter * 16, (3,3), activation = 'relu', padding = 'same', kernel_initializer='he_normal')(pool4)
    conv5_1 = Conv2D(initial_filter * 16, (3,3), activation = 'relu', padding = 'same', kernel_initializer='he_normal')(conv5)
    pool5 = MaxPool2D(pool_size = (2,2))(conv5_1)

    conv6 = Conv2D(initial_filter * 32, (3,3), activation = 'relu', padding = 'same', kernel_initializer='he_normal')(pool5)
    conv7 = Conv2D(initial_filter * 32, (3,3), activation = 'relu', padding = 'same', kernel_initializer='he_normal')(conv6)
    mul_up2 = UpSampling2D(size=(2,2))(conv7)
    mul_up = Conv2D(initial_filter * 8 ,(1,1))(mul_up2)
    FCN16s_add = tf.keras.layers.Add()([mul_up,pool4])

    up = UpSampling2D(size=(16,16))(FCN16s_add)
    out = Conv2D(1, (1,1))(up)

    model = Model(inputs, out)
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    if weight:
        model.load_weights(weight)


    return model

In [10]:

model = FCN16s()

In [11]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 8)  584         conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 128, 128, 8)  0           conv2d_1[0][0]                   
_______________________________________________________________________________________

In [ ]:
import math
history = model.fit_generator(trainSet_generator,100,epochs = 5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
 22/100 [=====>........................] - ETA: 2:12 - loss: 3.4739 - accuracy: 0.7621